In [ ]:
%load_ext autoreload
%autoreload 2
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
import os
import sys
import json
import random
module_path = os.path.abspath(os.path.join('../src')) # or the path to your source code
sys.path.insert(0, module_path)
import pandas
from query import query_cdp_data as query
from enrich import enrich_cdp_data as enrich
from ingest_bulk import ingest_cdp_data_bulk as ingest_bulk
from ingest_stream import ingest_cdp_data_stream as ingest_stream

In [ ]:
# update following
event = {}
event['login_url'] = 'https://login.salesforce.com' 
event['user_name'] = '' #CDP org's username
event['password'] = '' #CDP org's password
event['client_id'] = '' #CDP org's connected app client id
event['client_secret'] = '' #CDP org's connected app client secret
event['dlo_name'] = 'athlete_profiles' #CDP org's DLO name
event['dlo_source_name'] = 'athlete_api' #CDP org's Ingestion API source name 
event['dlo_object'] = 'athlete_api_athlete_profiles_E6B17A31__dll' #DLO object name to be queried
event['dlo_filter'] = 'limit 100000' #Where clause for the query
event['bulk_operation_type'] = 'upsert' #Where clause for the query

In [ ]:
#First load test data using streaming API
state = ["NY", "TX", "CT", "CA"]
athletes = []
athletes_insert_count = 5
for c in range(1, athletes_insert_count):
    i = random.randrange(100, 10000, 3)
    fname = f"fname-{i}"
    lname = f"lname-{i}"
    email = f"test-{i}@demp.cdp"
    
    a = {
    "maid":1010,
    "first_name":fname,
    "last_name":lname,
    "email":email,
    "gender":"Male",
    "city":"austin",
    "state":random.choice(state),
    "created":"2021-10-07T09:11:11.816319Z"
    }
    athletes.append(a)

ingest_stream(event, athletes)

In [ ]:
# Query DLO. # It can take 10 to 30 minutes for data to show up. Make sure to wait before you proceed to next step
df = query(event)
print(len(df))
df.head()

In [ ]:
# Enrich DLO
df = enrich(df)

In [ ]:
# Visualize enriched data
from matplotlib import pyplot as plt
plt.plot(df["state__c"], df["score"])
plt.show()

In [ ]:
# Bulk update Enriched DLO
ingest_bulk(event,df)

In [ ]:
df.head()